In [1]:
import matplotlib.pyplot as plt
from matplotlib import animation
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
class Particle: 
    def __init__(self, x, y, angular_velocity):
        self.x                = x
        self.y                = y
        self.angular_velocity = angular_velocity
    
    def __repr__(self):
        return 'Particle(%r, %r, omega=%r)' % (self.x, self.y, self.angular_velocity)

In [4]:
particle = Particle(0.0,0.1,3)
print(particle)
assert type(particle) is Particle

Particle(0.0, 0.1, omega=3)


In [5]:
particles = [Particle( 0.3,  0.5, +1),
             Particle( 0.0, -0.5, -1),
             Particle(-0.1, -0.4, +3)]

![](ang.png)

In [6]:
class ParticleSimulator:
    def __init__(self, particles):
        self.particles = particles
        
    def evolve(self, dt):
        timestep = 0.00001
        nsteps   = int(dt/timestep)
        
        for i in range(nsteps):
            for p in self.particles:
                #calculate the direction norm
                norm = (p.x**2 + p.y**2)**0.5
                v_x  = (-p.y)/norm
                v_y  = p.x/norm
                
                # calculate the displacement
                d_x = timestep * p.angular_velocity * v_x
                d_y = timestep * p.angular_velocity * v_y
                
                p.x += d_x
                p.y += d_y

In [7]:
simulator = ParticleSimulator(particles)

In [8]:
simulator.evolve(0.1)

In [9]:
p0, p1, p2 = particles

In [10]:
def fequal(a,b):
    return abs(a-b) < 1e-5

In [11]:
assert fequal(p0.x, 0.2102698450356825)
assert fequal(p0.y, 0.5438635787296997)
assert fequal(p1.x, -0.0993347660567358)
assert fequal(p1.y, -0.4900342888538049)
assert fequal(p2.x,  0.1913585038252641)
assert fequal(p2.y, -0.3652272210744360)

In [12]:
from tempfile import NamedTemporaryFile

VIDEO_TAG = """<video controls>
 <source src="data:video/x-m4v;base64,{0}" type="video/mp4">
 Your browser does not support the video tag.
</video>"""

def anim_to_html(anim):
    if not hasattr(anim, '_encoded_video'):
        with NamedTemporaryFile(suffix='.m4v') as f:
            anim.save(f.name, fps=20, extra_args=['-vcodec', 'libx264', '-pix_fmt', 'yuv420p'])
            video = open(f.name, "rb").read()
        anim.to_html5_video()
    # prevent figure displayed as a PNG below the animation
    plt.close()
    
    return VIDEO_TAG.format(anim.to_html5_video())

In [13]:
from IPython.display import HTML

def display_animation(anim):
    plt.close(anim._fig)
    return HTML(anim_to_html(anim))

In [16]:
X     = [p.x for p in simulator.particles]
Y     = [p.y for p in simulator.particles]

fig   = plt.figure()
ax    = plt.subplot(111, aspect='equal')
line, = ax.plot(X, Y, 'ro')

plt.xlim(-1,1)
plt.ylim(-1,1)

def init():
    line.set_data([], [])
    return line,

def animate(i):
    simulator.evolve(0.01)
    X = [p.x for p in simulator.particles]
    Y = [p.y for p in simulator.particles]

    line.set_data(X, Y)
    return line,

anim = animation.FuncAnimation(fig, animate, frames=1000,
                               init_func=init, interval=50,
                               blit=True)

display_animation(anim)

In [17]:
from random import uniform

def benchmark():
    particles = [Particle(uniform(-1.0, 1.0),
                          uniform(-1.0, 1.0),
                          uniform(-1.0, 1.0))
                  for i in range(1000)]
    
    simulator = ParticleSimulator(particles)
    simulator.evolve(0.1)

In [18]:
%timeit benchmark()

1 loop, best of 3: 9.7 s per loop


In [21]:
%prun benchmark()

In [23]:
%load_ext line_profiler

In [24]:
%lprun -f ParticleSimulator.evolve benchmark()

In [25]:
class ParticleSimulator:
    def __init__(self, particles):
        self.particles = particles
        
    def evolve(self, dt):
        timestep = 0.00001
        nsteps   = int(dt/timestep)
        
        for p in self.particles:
            t_x_ang = timestep * p.angular_velocity
            for i in range(nsteps):
            
                #calculate the direction norm
                norm = (p.x**2 + p.y**2)**0.5
                p.x, p.y = (p.x - t_x_ang * p.y/norm,
                              p.y + t_x_ang * p.x/norm)

In [26]:
%timeit benchmark()

1 loop, best of 3: 8.41 s per loop
